In [18]:
import os
import pandas as pd
import nibabel as nib
import numpy as np

import warnings
warnings.filterwarnings("ignore")

from skimage.measure import block_reduce
import numpy as np
from concurrent.futures import ProcessPoolExecutor
import time
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import tensorly as tl

#Debugging import
import importlib
var = 'TensorDecisionTreeRegressorP' #the published version of code
package = importlib.import_module(var)
for name, value in package.__dict__.items():
    if not name.startswith("__"):
        globals()[name] = value

from TensorDecisionTreeRegressorP import *

import os
import nibabel as nib
import numpy as np
import matplotlib as plt
import pandas as pd
from sklearn.model_selection import train_test_split


# File path to the CSV file
csv_file = '/Users/zhihaochen/Documents/CommenDesktop/RICE/MyProject/Bayes_Tensor_Tree/3D-images/ADNIData.csv'
df = pd.read_csv(csv_file)

# Remove rows where ADAS11_bl is missing (NaN)
df_cleaned = df.dropna(subset=['RAVLT_forgetting'])

# Extract the 'ADAS11_bl' column as the y variable
y_variable = df_cleaned['RAVLT_forgetting'].values

# Split the dataframe based on the DX_bl column values
cn_group = df_cleaned[df_cleaned['DX_bl'] == 'CN']
ad_group = df_cleaned[df_cleaned['DX_bl'] == 'AD']
lmci_group = df_cleaned[df_cleaned['DX_bl'] == 'LMCI']

# Display the counts for each group after removing NA
print(f"CN group size: {cn_group.shape[0]}")
print(f"AD group size: {ad_group.shape[0]}")
print(f"LMCI group size: {lmci_group.shape[0]}")

# Directory containing the 3D images
directory = '/Users/zhihaochen/Documents/CommenDesktop/RICE/MyProject/Bayes_Tensor_Tree/3D-images/3D-Images/bl'

# Initialize dictionaries to hold the images and y values for each group
cn_images, ad_images, lmci_images = [], [], []
cn_y, ad_y, lmci_y = [], [], []

# Function to load the images based on PTID matching and append y values
def load_images_and_y(group, image_list, y_list):
    for _, row in group.iterrows():
        ptid = row['PTID']
        # Find the corresponding file based on PTID
        filename = f'{ptid}.nii.gz'
        file_path = os.path.join(directory, filename)
        
        if os.path.exists(file_path):
            # Load the NIfTI file
            img = nib.load(file_path)
            data = img.get_fdata()
            
            # Append the 3D image data and y value to the respective lists
            image_list.append(data)
            y_list.append(row['RAVLT_forgetting'])
        else:
            print(f"File {filename} not found.")

# Load images and y values for each group
load_images_and_y(cn_group, cn_images, cn_y)
load_images_and_y(ad_group, ad_images, ad_y)
load_images_and_y(lmci_group, lmci_images, lmci_y)

# Convert lists of 3D images and y values to NumPy arrays
if cn_images:
    cn_tensor = np.stack(cn_images, axis=0)
    cn_y = np.array(cn_y)
    print(f"CN 4D tensor shape: {cn_tensor.shape}")
    print(f"CN y shape: {cn_y.shape}")
else:
    print("No CN images loaded.")

if ad_images:
    ad_tensor = np.stack(ad_images, axis=0)
    ad_y = np.array(ad_y)
    print(f"AD 4D tensor shape: {ad_tensor.shape}")
    print(f"AD y shape: {ad_y.shape}")
else:
    print("No AD images loaded.")

if lmci_images:
    lmci_tensor = np.stack(lmci_images, axis=0)
    lmci_y = np.array(lmci_y)
    print(f"LMCI 4D tensor shape: {lmci_tensor.shape}")
    print(f"LMCI y shape: {lmci_y.shape}")
else:
    print("No LMCI images loaded.")



CN group size: 227
AD group size: 186
LMCI group size: 401
CN 4D tensor shape: (227, 48, 48, 48)
CN y shape: (227,)
AD 4D tensor shape: (186, 48, 48, 48)
AD y shape: (186,)
LMCI 4D tensor shape: (401, 48, 48, 48)
LMCI y shape: (401,)


In [11]:
const_array = np.load('const_list.npy', allow_pickle=True)
const_list = list(map(tuple, const_array))


X_train, X_test, y_train, y_test = train_test_split(lmci_tensor, lmci_y, test_size=0.3, random_state=42)
print(X_train.shape,y_train.shape)
model  =  TensorDecisionTreeRegressor(max_depth=10, min_samples_split=12,split_method='variance_LS', split_rank=4, CP_reg_rank=3, Tucker_reg_rank=3, n_mode=4, const_array=const_list)
model.sample_rate  =  .1
model.fit(X_train,y_train)

predictions = model.predict(X_train,regression_method='mean')
print(f"mean train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train))
predictions = model.predict(X_train,regression_method='cp')
print(f"CP train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train))
predictions = model.predict(X_train,regression_method='tucker')
print(f"Tucker train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train)) 

predictions = model.predict(X_test,regression_method='mean')
print(f"mean test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))
predictions = model.predict(X_test,regression_method='cp')
print(f"CP test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))
predictions = model.predict(X_test,regression_method='tucker')
print(f"Tucker test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))

(280, 48, 48, 48) (280,)
mean train RSE:  0.13582956334548887
CP train RSE:  0.0002180459691911054
Tucker train RSE:  0.0002735394501519964
mean test RSE:  1.6686879726107033
CP test RSE:  2.2068048900623687
Tucker test RSE:  2.038033863390594


In [16]:
model.print_tree()

  has  0  child nodes, and  280  samples.


In [17]:
model.count_leaves()

1

In [14]:
model.prune(alpha = 0.000000000000001)

In [15]:
model.print_tree()

  has  0  child nodes, and  280  samples.


In [23]:
os.listdir(directory)

['098_S_0269.nii.gz',
 '002_S_0619.nii.gz',
 '029_S_0999.nii.gz',
 '052_S_0951.nii.gz',
 '068_S_0109.nii.gz',
 '133_S_0771.nii.gz',
 '127_S_1419.nii.gz',
 '057_S_0474.nii.gz',
 '003_S_1059.nii.gz',
 '128_S_0611.nii.gz',
 '137_S_0481.nii.gz',
 '136_S_0695.nii.gz',
 '032_S_0214.nii.gz',
 '041_S_1418.nii.gz',
 '094_S_0692.nii.gz',
 '098_S_0884.nii.gz',
 '036_S_1135.nii.gz',
 '011_S_0003.nii.gz',
 '006_S_0731.nii.gz',
 '128_S_1406.nii.gz',
 '013_S_1161.nii.gz',
 '062_S_1294.nii.gz',
 '033_S_1309.nii.gz',
 '006_S_0498.nii.gz',
 '023_S_0126.nii.gz',
 '082_S_0928.nii.gz',
 '098_S_0896.nii.gz',
 '082_S_0640.nii.gz',
 '082_S_1377.nii.gz',
 '018_S_0087.nii.gz',
 '116_S_1271.nii.gz',
 '123_S_0106.nii.gz',
 '021_S_0984.nii.gz',
 '002_S_0955.nii.gz',
 '035_S_0204.nii.gz',
 '035_S_0555.nii.gz',
 '128_S_1430.nii.gz',
 '073_S_0909.nii.gz',
 '009_S_0862.nii.gz',
 '032_S_0187.nii.gz',
 '100_S_0190.nii.gz',
 '031_S_0351.nii.gz',
 '133_S_0493.nii.gz',
 '023_S_0061.nii.gz',
 '014_S_0558.nii.gz',
 '013_S_02